# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction/Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction/Business Problem <a name="introduction"></a>

In this project we will identify **neighborhoods in Toronto, Canada and New York City, USA that fit** the international **expansion** strategy **for an Italian restaurant chain based in Milan, Italy.**  The primary focus of this strategy is to **identify neighborhoods that have a similar commercial venue profile to the Milan neighborhoods** that the chain and other Italian resturants currently operate in. 

Following **this strategy** offers several benefits to the chain as it seeks to establish a presence on the east coast of North America.  It **reduces the learning curve of operating in new countries by reducing the unknowns.**  The country will still be unfamiliar however the neighborhood profiles will be familiar and more normal to operate in.  In addition, by identifying neighborhoods that already support Italian restaurants, the chain will be able to **benefit from the "second mover" advantage** as it is already established that Italian neighborhoods can be successful there.  

The neighborhoods that are identified in this project will be utilized to support future expansion decisions by the chain.  The stakeholders who will benefit the most from this project are the members of the chain's management team that has been tasked with North America expansion.

## Data <a name="data"></a>

To successfully complete this project, it will be neccessary to identify:
* Neighborhood names and commercial venue profiles of Milan neighborhoods where Italian restaurants are successful
* Neighborhood names and commercial venue profiles of similar Toronto and New York City neighborhoods

The raw data neccessary to complete these tasks will be identified and scraped from across the internet.  Since this will be a comprehensive analysis that covers all neighborhoods of the three cities, it will require a large amount of raw data that will consume the single computer resources available for wrangling and processing.  To make the work manageable, it was decided to segment the work for each of the individual cities as much as possible then to concatenate the data for the final analysis.

The **Milan** borough and neighborhood names were web scraped utilizing Beautiful Soup from the Wikipedia web page: [Municipalities_of_Milan](https://en.wikipedia.org/wiki/Municipalities_of_Milan).  This data was wrangled into the proper format then geopy was utilized to obtain the geospatial coordinates for each neighborhood.  This information was then passed to the Foursquare API which provided the top 100 venues within a 500 meter radius from the geospatial coordinates of each of the 134 neighborhoods of Milan.  After further data wrangling, this data was then ready to be analyzed.

The **Toronto** borough and neighborhood names along with their corresponding geospatial coordinates were obtained from a previous project in this course.  It is noted that the previous project on Toronto limited the neighborhoods to only those containing "Toronto" in the name whereas this report covers all Toronto neighborhoods thus additional data mining was required.  This data was wrangled into the proper format then passed to the Foursquare API which provided the top 100 venues within a 500 meter radius from the geospatial coordinates of each of the 103 neighborhoods of Toronto.  After further data wrangling, this data was then ready to be analyzed.

The **New York City** borough and neighborhood names along with their corresponding geospatial coordinates were obtained from a previous project in this course.  It is noted that the previous project on New York City limited the neighborhoods to only those in the Manhattan borough whereas this report covers all New York City neighborhoods in all five boroughs thus additional data mining was required.  This data was wrangled into the proper format then passed to the Foursquare API which provided the top 100 venues within a 500 meter radius from the geospatial coordinates of each of the 306 neighborhoods of New York City.  After further data wrangling, this data was then ready to be analyzed.

### Notebook Preparation

Importing the libraries and modules neccessary to complete this project

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import folium # map rendering library

import wget

import geopy
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

from urllib.request import urlopen

from collections import Counter

print('Libraries imported.')


Libraries imported.


### Milan, Italy

In [1]:
url = 'https://en.wikipedia.org/wiki/Municipalities_of_Milan'

In [3]:
html = urlopen(url)

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [5]:
table =soup.select('table')[-2]

In [6]:
#Create array to hold the data we extract
boroughs = []
neighborhoods = []

rows = table.find_all('tr')

for row in rows:
    cells = row.find_all('td')
        
        
    if len(cells) > 1:
        borough = cells[1]
        boroughs.append(borough.text.strip())
            
        neighborhood = cells[5]
        neighborhoods.append(neighborhood.text.strip())

# for table in tables:
    # rows = table.find_all('tr')
    
    # for row in rows:
        # cells = row.find_all('td')
        
        
        # if len(cells) > 1:
            # borough = cells[1]
            # boroughs.append(borough.text.strip())
            
            # neighborhood = cells[5]
            # neighborhoods.append(neighborhood.text.strip())

In [7]:
ml_df1 = pd.DataFrame(boroughs, columns = ['Borough'])
ml_df1['Neighborhood'] = neighborhoods

print(ml_df1.shape)
ml_df1.head(10)

(9, 2)


,Borough,Neighborhood
0,Centro storico,"Brera, Centro Storico, Conca del Naviglio, Gua..."
1,"Stazione Centrale, Gorla, Turro, Greco, Cresce...","Adriano, Crescenzago, Gorla, Greco, Loreto, Ma..."
2,"Città Studi, Lambrate, Porta Venezia","Casoretto, Cimiano, Città Studi, Dosso, Lambra..."
3,"Porta Vittoria, Forlanini","Acquabella, Calvairate, Castagnedo, Cavriano, ..."
4,"Vigentino, Chiaravalle, Gratosoglio","Basmetto, Cantalupa, Case Nuove, Chiaravalle, ..."
5,"Barona, Lorenteggio","Arzaga, Barona, Boffalora, Cascina Bianca, Con..."
6,"Baggio, De Angeli, San Siro","Assiano, Baggio, Figino, Fopponino, Forze Arma..."
7,"Fiera, Gallaratese, Quarto Oggiaro","Boldinasco, Bullona, Cagnola, Campo dei Fiori,..."
8,"Porta Garibaldi, Niguarda","Affori, Bicocca, Bovisa, Bovisasca, Bruzzano, ..."


In [8]:
def split_df_rows_for_comma_separated_key(key, df):
    df=df.set_index(df.columns.drop(key,1).tolist())[key].str.split(',', expand=True).stack().reset_index().rename(columns={0:key}).loc[:, df.columns]
    df=df[df[key] != '']
    return df

In [10]:
ml_df2 = split_df_rows_for_comma_separated_key('Neighborhood', ml_df1)

# Remove the blank space at the beginning of the neighborhood names
ml_df2['Neighborhood'] = ml_df2['Neighborhood'].str.strip()

print(ml_df2.shape)
ml_df2.head(10)

(145, 2)


,Borough,Neighborhood
0,Centro storico,Brera
1,Centro storico,Centro Storico
2,Centro storico,Conca del Naviglio
3,Centro storico,Guastalla
4,Centro storico,Porta Sempione
5,Centro storico,Porta Tenaglia
6,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Adriano
7,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Crescenzago
8,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Gorla
9,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Greco


In [13]:
geolocator = Nominatim(user_agent = 'myGeocoder')

# Conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

ml_df2['Location'] = ml_df2['Neighborhood'] + ', Milan'
# ml_df2['Location'] = ml_df2['Neighborhood']

ml_df2['gcode'] = ml_df2.Location.apply(geolocator.geocode)

ml_df2['Latitude'] = [g.latitude for g in ml_df2.gcode]
ml_df2['Longitude'] = [g.longitude for g in ml_df2.gcode]

ml_df3 = ml_df2.drop(['Location', 'gcode'], axis = 1)

print(ml_df3.shape)
ml_df3.head(10)

(145, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Centro storico,Brera,45.473479,9.188408
1,Centro storico,Centro Storico,45.447112,9.094054
2,Centro storico,Conca del Naviglio,45.458560,9.177745
3,Centro storico,Guastalla,45.458252,9.200023
4,Centro storico,Porta Sempione,45.477128,9.170598
5,Centro storico,Porta Tenaglia,45.477821,9.181593
6,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Adriano,45.513572,9.251202
7,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Crescenzago,45.509219,9.247484
8,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Gorla,45.504945,9.224539
9,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Greco,45.502184,9.211233


In [14]:
# Made corrections to eliminate outliers based on generating the below map of Milan and observing outliers which are presumed to be due
# which are presumed to be due to innaccurate geospatial coordinates for a few neighborhoods.

lon_correction1 = ml_df3[(ml_df3['Longitude'] > 9.3)].index
ml_df4 = ml_df3.drop(lon_correction1)

lon_correction2 = ml_df4[(ml_df4['Longitude'] < 9.05)].index
ml_df4.drop(lon_correction2, inplace = True)

lat_correction1 = ml_df4[(ml_df4['Latitude'] < 45.3)].index
ml_df4.drop(lat_correction1, inplace = True)

lat_correction2 = ml_df4[(ml_df4['Latitude'] > 45.6)].index
ml_df4.drop(lat_correction2, inplace = True)

ml_df4.index = range(len(ml_df4.index))

print(ml_df4.shape)
ml_df4.head(10)

(136, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Centro storico,Brera,45.473479,9.188408
1,Centro storico,Centro Storico,45.447112,9.094054
2,Centro storico,Conca del Naviglio,45.458560,9.177745
3,Centro storico,Guastalla,45.458252,9.200023
4,Centro storico,Porta Sempione,45.477128,9.170598
5,Centro storico,Porta Tenaglia,45.477821,9.181593
6,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Adriano,45.513572,9.251202
7,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Crescenzago,45.509219,9.247484
8,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Gorla,45.504945,9.224539
9,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",Greco,45.502184,9.211233


In [15]:
# Use geopy library to get the latitude and longitude values of Milan

address = 'Milan, Italy'

geolocator = Nominatim(user_agent="mil_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milan are 45.4668, 9.1905.


In [18]:
# create map of Milan with the neighborhoods superimposed on it using latitude and longitude values  
map_milan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(ml_df4['Latitude'], ml_df4['Longitude'], ml_df4['Borough'], ml_df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_milan)  
    
map_milan

Define Foursquare Credentials and Version

In [19]:
# The code was removed by Watson Studio for sharing.

In [22]:
# The code was removed by Watson Studio for sharing.

In [23]:
# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\shive\anaconda3\envs\IBM Classes\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Pinacoteca di Brera,Art Museum,45.471979,9.188128
1,Palazzo Parigi,Hotel,45.473378,9.191108
2,Palazzo di Brera,College Arts Building,45.472019,9.188043
3,Hops Beer Shop Milano,Beer Store,45.475240,9.189126
4,SUSHI B,Japanese Restaurant,45.472153,9.186883


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

98 venues were returned by Foursquare.


In [36]:
# The code was removed by Watson Studio for sharing.

In [29]:
print(milan_venues.shape)
milan_venues.head(10)

(3759, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Brera,45.473479,9.188408,Pinacoteca di Brera,45.471979,9.188128,Art Museum
1,Brera,45.473479,9.188408,Palazzo Parigi,45.473378,9.191108,Hotel
2,Brera,45.473479,9.188408,Palazzo di Brera,45.472019,9.188043,College Arts Building
3,Brera,45.473479,9.188408,Hops Beer Shop Milano,45.475240,9.189126,Beer Store
4,Brera,45.473479,9.188408,SUSHI B,45.472153,9.186883,Japanese Restaurant
5,Brera,45.473479,9.188408,Dimore Studio,45.474770,9.186954,Art Gallery
6,Brera,45.473479,9.188408,Osteria di Brera,45.472207,9.187045,Italian Restaurant
7,Brera,45.473479,9.188408,Di Viole Di Liquirizia,45.471460,9.185336,Cupcake Shop
8,Brera,45.473479,9.188408,Orto Botanico di Brera,45.471197,9.189577,Botanical Garden
9,Brera,45.473479,9.188408,BVLGARI Hotel Milano,45.470149,9.189318,Hotel


In [30]:
milan_venues.groupby('Neighborhood').count().head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acquabella,7,7,7,7,7,7
Adriano,8,8,8,8,8,8
Affori,16,16,16,16,16,16
Arzaga,22,22,22,22,22,22
Assiano,2,2,2,2,2,2
Baggio,10,10,10,10,10,10
Barona,7,7,7,7,7,7
Bicocca,38,38,38,38,38,38
Boffalora,8,8,8,8,8,8


In [31]:
print('There are {} uniques categories.'.format(len(milan_venues['Venue Category'].unique())))

There are 298 uniques categories.


### Toronto, Canada

The Toronto dataframe containing the neighborhood information and geospatial coordinates was already created for a previous project for this course therefore instead of recreating it, the magic store command will be utilized to bring the dataframe from its original notebook into this notebook.

In [32]:
%store -r WT3

print(WT3.shape)
WT3.head(10)

(103, 5)


,City,Borough,Neighborhood,Latitude,Longitude
0,Toronto,North York,Parkwoods,43.753259,-79.329656
1,Toronto,North York,Victoria Village,43.725882,-79.315572
2,Toronto,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,Toronto,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Toronto,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,Toronto,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,Toronto,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,Toronto,North York,Don Mills,43.745906,-79.352188
8,Toronto,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,Toronto,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [34]:
# Use geopy library to get the latitude and longitude values of Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [35]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(WT3['Latitude'], WT3['Longitude'], WT3['Borough'], WT3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(2167, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [38]:
toronto_venues.groupby('Neighborhood').count().head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [39]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


### New York City, USA

The New York City dataframe containing the neighborhood information and geospatial coordinates was already created for a previous project for this course therefore instead of recreating it, the magic store command will be utilized to bring the dataframe from its original notebook into this notebook.

In [40]:
%store -r NYC_df

print(NYC_df.shape)
NYC_df.head(10)

(306, 5)


,City,Borough,Neighborhood,Latitude,Longitude
0,New York City,Bronx,Wakefield,40.894705,-73.847201
1,New York City,Bronx,Co-op City,40.874294,-73.829939
2,New York City,Bronx,Eastchester,40.887556,-73.827806
3,New York City,Bronx,Fieldston,40.895437,-73.905643
4,New York City,Bronx,Riverdale,40.890834,-73.912585
5,New York City,Bronx,Kingsbridge,40.881687,-73.902818
6,New York City,Manhattan,Marble Hill,40.876551,-73.910660
7,New York City,Bronx,Woodlawn,40.898273,-73.867315
8,New York City,Bronx,Norwood,40.877224,-73.879391
9,New York City,Bronx,Williamsbridge,40.881039,-73.857446


In [41]:
# Use geopy library to get the latitude and longitude values of New York City

address = 'New York City'

geolocator = Nominatim(user_agent="nyc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [42]:
# create map of New York City using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NYC_df['Latitude'], NYC_df['Longitude'], NYC_df['Borough'], NYC_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

In [43]:
print(NYC_venues.shape)
NYC_venues.head()

(10478, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop


In [44]:
NYC_venues.groupby('Neighborhood').count().head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,34,34,34,34,34,34
Annadale,12,12,12,12,12,12
Arden Heights,6,6,6,6,6,6
Arlington,7,7,7,7,7,7
Arrochar,24,24,24,24,24,24
Arverne,21,21,21,21,21,21
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,20,20,20,20,20,20


In [45]:
print('There are {} uniques categories.'.format(len(NYC_venues['Venue Category'].unique())))

There are 436 uniques categories.


## Methodology <a name="methodology"></a>

## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>